In [1]:
# import os

# def count_yolo_classes(label_dir):
#     class_ids = set()
    
#     for filename in os.listdir(label_dir):
#         if filename.endswith(".txt"):
#             filepath = os.path.join(label_dir, filename)
#             with open(filepath, 'r') as f:
#                 for line in f:
#                     parts = line.strip().split()
#                     if parts:  # avoid empty lines
#                         class_id = int(parts[0])
#                         class_ids.add(class_id)
    
#     print(f"Number of unique classes: {len(class_ids)}")
#     print(f"Classes found: {sorted(class_ids)}")

# directory_path = "/kaggle/input/rdd-2022/RDD_SPLIT/train/labels"
# count_yolo_classes(directory_path)

Number of unique classes: 5
Classes found: [0, 1, 2, 3, 4]


In [2]:
# import os
# from collections import Counter

# labels_dir = '/kaggle/input/rdd-2022/RDD_SPLIT/train/labels'
# all_classes = []

# for label_file in os.listdir(labels_dir):
#     if not label_file.endswith('.txt'):
#         continue
#     with open(os.path.join(labels_dir, label_file), 'r') as f:
#         for line in f:
#             class_id = int(line.strip().split()[0])
#             all_classes.append(class_id)

# print(Counter(all_classes))

Counter({0: 18201, 1: 8386, 3: 7554, 2: 7527, 4: 4628})


In [3]:
# import os
# import shutil
# import pandas as pd

# # Define class mappings (human-readable damage labels)
# class_id_to_name = {
#     0: 'longitudinal_crack',
#     1: 'transverse_crack',
#     2: 'alligator_crack',
#     3: 'pothole',
#     4: 'other_damage'
# }

# class_names = list(class_id_to_name.values())
# num_classes = len(class_names)

# # Dataset paths
# original_base = '/kaggle/input/rdd-2022/RDD_SPLIT'
# splits = ['train', 'val', 'test']
# new_base = '/kaggle/working/'

# # Loop through the splits (train, val, test) to process CSVs
# for split in splits:
#     labels_dir = os.path.join(original_base, split, 'labels')
#     data = []

#     # Process each label file in the 'labels' directory
#     for label_file in os.listdir(labels_dir):
#         if not label_file.lower().endswith('.txt'):
#             continue

#         label_path = os.path.join(labels_dir, label_file)
#         label_vector = [0] * num_classes

#         with open(label_path, 'r') as f:
#             for line in f:
#                 class_id = int(line.strip().split()[0])
#                 if class_id in class_id_to_name:
#                     label_vector[class_id] = 1

#         # Assuming you have a naming convention for image files (e.g., the label file name matches the image file name)
#         img_file = label_file.replace('.txt', '.jpg')  # or .png, .jpeg based on your dataset format
#         data.append([img_file] + label_vector)

#     # Save the CSV file for the current split
#     df = pd.DataFrame(data, columns=['filename'] + class_names)
#     df.to_csv(os.path.join(new_base, f'{split}_labels.csv'), index=False)


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.losses import BinaryFocalCrossentropy
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score

# Constants
new_base = '/kaggle/input/rdd-2022/RDD_SPLIT'
csv_base = '/kaggle/working'
batch_size = 32
target_size = (224, 224)

# Class names
class_names = ['longitudinal_crack', 'transverse_crack', 'alligator_crack', 'pothole', 'other_damage']
num_classes = len(class_names)

# Load data
train_df = pd.read_csv(os.path.join(csv_base, 'train_labels.csv'))
val_df = pd.read_csv(os.path.join(csv_base, 'val_labels.csv'))
test_df = pd.read_csv(os.path.join(csv_base, 'test_labels.csv'))

# Compute class weights from integer labels
y_train = train_df[class_names].values
y_train_classes = np.argmax(y_train, axis=1)
weights = compute_class_weight(class_weight='balanced', classes=np.arange(num_classes), y=y_train_classes)
class_weight_dict = {i: weights[i] for i in range(num_classes)}

# Image generator
def create_datagen(df, directory, batch_size, target_size, class_names, shuffle=False):
    def generator():
        indices = df.index.tolist()
        if shuffle:
            np.random.shuffle(indices)
        for i in indices:
            row = df.loc[i]
            img_path = os.path.join(directory, row['filename'])
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            label = row[class_names].values.astype('float32')
            yield img_array, label

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([*target_size, 3], [len(class_names)])
    )
    return dataset.batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)

# Dataset paths
train_dir = os.path.join(new_base, 'train/images')
val_dir = os.path.join(new_base, 'val/images')
test_dir = os.path.join(new_base, 'test/images')

# Datasets
train_ds = create_datagen(train_df, train_dir, batch_size, target_size, class_names, shuffle=True)
val_ds = create_datagen(val_df, val_dir, batch_size, target_size, class_names)
test_ds = create_datagen(test_df, test_dir, batch_size, target_size, class_names)

steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size
test_steps = len(test_df) // batch_size

# Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# Model builder
def build_model(input_shape, num_classes):
    base_model = EfficientNetB3(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    return models.Model(inputs, outputs)

# Loss and callbacks
focal_loss = BinaryFocalCrossentropy(gamma=2.0)
checkpoint_cb = callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_binary_accuracy', mode='max')
early_stop_cb = callbacks.EarlyStopping(patience=6, restore_best_weights=True)

# Compile model
model = build_model((224, 224, 3), num_classes)
model.compile(optimizer='adam', loss=focal_loss, metrics=['binary_accuracy'])

# Train - Phase 1
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint_cb, early_stop_cb],
    class_weight=class_weight_dict
)

# Unfreeze last N layers for fine-tuning
base_model = model.get_layer(index=2)
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=focal_loss, metrics=['binary_accuracy'])

# Train - Phase 2
fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint_cb]
)

# Evaluate
model.load_weights('best_model.keras')
test_loss, test_acc = model.evaluate(test_ds.take(test_steps))
print(f"Final Test Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")

# Classification report
y_true = test_df[class_names].values
y_pred_probs = model.predict(test_ds.take(test_steps))
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:\n", classification_report(y_true[:len(y_pred)], y_pred, target_names=class_names))
print("Macro F1 Score:", f1_score(y_true[:len(y_pred)], y_pred, average='macro'))


I0000 00:00:1745531460.299903      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745531460.300542      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


E0000 00:00:1745531488.018729      31 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1745531491.172566     100 cuda_dnn.cc:529] Loaded cuDNN version 90300


839/839 ━━━━━━━━━━━━━━━━━━━━ 630s 718ms/step - binary_accuracy: 0.7563 - loss: 0.1450 - val_binary_accuracy: 0.7905 - val_loss: 0.1388
Epoch 2/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 425s 507ms/step - binary_accuracy: 0.7650 - loss: 0.1444 - val_binary_accuracy: 0.7907 - val_loss: 0.1503
Epoch 3/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 413s 493ms/step - binary_accuracy: 0.7678 - loss: 0.1440 - val_binary_accuracy: 0.7910 - val_loss: 0.1427
Epoch 4/10
256/839 ━━━━━━━━━━━━━━━━━━━━ 3:44 385ms/step - binary_accuracy: 0.7717 - loss: 0.1431